In [1]:
import numpy as np
import os
import sys
from pathlib import Path

module_path = str(Path(os.path.abspath('')).parent)
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.sac_modular import *
from functions.RNCRN_tools import * 

In [2]:
# creates the classification-controlled RNCRN from the two subsystem models i.e. point-wise parametrized RNCRN & Quasi-static classification CRN
# set initial conditions for CRN (overwritten later)
env_inits = [0.5]
neuron_classify_inits = [0, 0, 0]
bif_inits = [0]
exec_species = [1, 1]
neuron_toggle_inits = [0, 0, 0, 0, 0]

# set timescale for both subsystems
time_scale_classify = 0.01
time_scale_toggle = 0.01

# filenames of subsystem models 
filename_classify = r'models/example_classify_subsystem'
filename_toggle = r'models/example_para_RNCRN'

# output location of the compiled classification-controlled RNCRN (as a CRN)
crn_file = 'CRNs/data_def_piecewise_eq_cycle_crn.txt'

# function to compile models into a chemical reaction network 
CRN = create_sense_and_respond_CRN(filename_classify, filename_toggle, time_scale_classify, time_scale_toggle , env_inits, neuron_classify_inits, bif_inits,exec_species, neuron_toggle_inits)
save_crn(crn_file, CRN )
print(CRN)

# loads the chemical reaction network file from .txt and extracts the kinetic and stoichiometric data
species, reaction_rates, react_stoch, prod_stoch, stoch_mat, number_species, number_reactions, initial_concs = read_crn_txt(crn_file)

# creates a tuple of kinetic and stoichiometric data for the trajectory simulation 
args_tup_rncrn = (reaction_rates, react_stoch, stoch_mat,)

#L_1=0.5,Z_1=0,Z_2=0,Z_3=0,R_1=0
L_1 + Z_1->L_1,389.91172313690186
L_1 + Z_2->L_1,483.17790031433105
L_1 + Z_3->L_1,508.870792388916
Z_1->Z_1 + Z_1,1078.57084274292
Z_2->Z_2 + Z_2,1186.4456176757812
Z_3->Z_3 + Z_3,1252.7347564697266
->Z_1,100.0
->Z_2,100.0
->Z_3,100.0
Z_1 + Z_1->Z_1,100.0
Z_2 + Z_2->Z_2,100.0
Z_3 + Z_3->Z_3,100.0
Z_1 + R_1->Z_1 + R_1 + R_1,596.0429668426514
Z_2 + R_1->Z_2,525.4505634307861
Z_3 + R_1->Z_3,750.502347946167
R_1->R_1 + R_1,43.5576856136322
->R_1,100.0
R_1 + R_1->R_1,100.0
R_1 + Y_1->R_1 + Y_1 + Y_1,238034.27734375
R_1 + Y_2->R_1 + Y_2 + Y_2,160515.39306640625
R_1 + Y_3->R_1 + Y_3 + Y_3,119853.02734375
R_1 + Y_4->R_1 + Y_4 + Y_4,273762.40234375
R_1 + Y_5->R_1,838222.8515625
#X_1=1,X_2=1,Y_1=0,Y_2=0,Y_3=0,Y_4=0,Y_5=0
X_1 + Y_1->Y_1 + X_1 + X_1,89.51668
X_1 + Y_2->Y_2,13.080138
X_1 + Y_3->Y_3 + X_1 + X_1,0.32079333
X_1 + Y_4->Y_4,98.54018
X_1 + Y_5->Y_5,69.70664
X_2 + Y_1->Y_1 + X_2 + X_2,96.383896
X_2 + Y_2->Y_2,50.70877
X_2 + Y_3->Y_3,24.35277
X_2 + Y_4->Y_

In [3]:
# numerical integration of the RNCRN 
# for numerical integration of full classification-controlled RNCRN
vals = np.arange(0, 6, 0.5)
num_graphs = len(vals)

# the time length of the trajectory
tFinal = 25
tRange = np.arange(0, tFinal, 0.01)

for i in np.arange(0, num_graphs, 1):
    lambda_1 = vals[i]
    print(i, lambda_1)    
    # checks if data already created. If not, then runs numerical integration
    try:
        t_low = np.load('data/sol_rncrn_low_t_i'+str(i)+'.npy')
        y_low = np.load('data/sol_rncrn_low_y_i'+str(i)+'.npy')
    except:
        inits_rncrn_low, id_dict_low = new_initial_conditions(initial_concs, species, {'L_1':lambda_1, 'X_1': 1.0, 'X_2': 1.0})
        sol_rncrn_low = solve_ivp(stoch_mat_to_mass_action, [0, tFinal], inits_rncrn_low, args=args_tup_rncrn, rtol=10e-8, t_eval=tRange)
        np.save('data/sol_rncrn_low_t_i'+str(i)+'.npy', sol_rncrn_low.t)
        np.save('data/sol_rncrn_low_y_i'+str(i)+'.npy', sol_rncrn_low.y[id_dict_low['X_1'],:])

0 0.0
1 0.5
2 1.0
3 1.5
4 2.0
5 2.5
6 3.0
7 3.5
8 4.0
9 4.5
10 5.0
11 5.5
